In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Define Weights for Credit Score Calculation
weights = {
    "DTI": -0.35,  # High DTI is bad
    "STI": 0.25,   # High savings is good
    "CUR": -0.30,  # High credit utilization is bad
    "TOTAL_SPENDING": -0.10,  # High spending can be risky
    "GAMBLING_PERCENTAGE": -0.40,  # High gambling is very bad
    "IS_HIGH_DEBT": -0.50,  # High debt means high risk
    "IS_LOW_SAVINGS": -0.30  # Low savings is risky
}

In [3]:
def compute_credit_score(data):
    """
    Takes user input, calculates financial metrics, and computes a credit score.
    """
    # Extract values
    income = data["INCOME"]
    savings = data["SAVINGS"]
    debt = data["DEBT"]

    # Spending categories
    spending_categories = ["T_GROCERIES_12", "T_CLOTHING_12", "T_HOUSING_12", "T_EDUCATION_12", 
                           "T_HEALTH_12", "T_TRAVEL_12", "T_ENTERTAINMENT_12", "T_GAMBLING_12", 
                           "T_UTILITIES_12", "T_TAX_12", "T_FINES_12"]

    total_spending = sum(data[cat] for cat in spending_categories)

    # Compute Derived Features
    DTI = debt / income if income > 0 else 1  # Avoid division by zero
    STI = savings / income if income > 0 else 0
    CUR = debt / (debt + savings + income) if (debt + savings + income) > 0 else 1
    GAMBLING_PERCENTAGE = data["T_GAMBLING_12"] / total_spending if total_spending > 0 else 0
    IS_HIGH_DEBT = 1 if DTI > 0.5 else 0
    IS_LOW_SAVINGS = 1 if STI < 0.2 else 0

    # Normalize the features before scoring
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(np.array([[DTI, STI, CUR, total_spending, GAMBLING_PERCENTAGE, IS_HIGH_DEBT, IS_LOW_SAVINGS]]))

    # Compute Weighted Credit Score
    credit_score = (
        scaled_features[0][0] * weights["DTI"] +
        scaled_features[0][1] * weights["STI"] +
        scaled_features[0][2] * weights["CUR"] +
        scaled_features[0][3] * weights["TOTAL_SPENDING"] +
        scaled_features[0][4] * weights["GAMBLING_PERCENTAGE"] +
        scaled_features[0][5] * weights["IS_HIGH_DEBT"] +
        scaled_features[0][6] * weights["IS_LOW_SAVINGS"]
    )

    # Scale Credit Score to 300 - 850 range
    scaled_score = MinMaxScaler(feature_range=(300, 850)).fit_transform([[credit_score]])[0][0]
    final_credit_score = round(scaled_score, 2)

    # Loan Approval Decision
    loan_status = "APPROVED" if final_credit_score >= 700 else "REJECTED"

    return {
        "credit_score": final_credit_score,
        "loan_status": loan_status,
        "financial_metrics": {
            "DTI": round(DTI, 2),
            "STI": round(STI, 2),
            "CUR": round(CUR, 2),
            "TOTAL_SPENDING": round(total_spending, 2),
            "GAMBLING_PERCENTAGE": round(GAMBLING_PERCENTAGE, 2),
            "IS_HIGH_DEBT": IS_HIGH_DEBT,
            "IS_LOW_SAVINGS": IS_LOW_SAVINGS
        }
    }